## MODEL- FEATURE1 - IMAGE UPLOAD FROM GALLERY

In [6]:
import torch
import timm
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import cv2
import numpy as np

# Load Pretrained Xception Model
class DeepfakeDetector(nn.Module):
    def __init__(self):
        super(DeepfakeDetector, self).__init__()
        self.model = timm.create_model('xception', pretrained=True)  # Load Xception
        self.model.fc = nn.Linear(self.model.num_features, 1)  # Modify output for binary classification
        self.sigmoid = nn.Sigmoid()  # Sigmoid for probability score

    def forward(self, x):
        return self.sigmoid(self.model(x))

# Load Model
model = DeepfakeDetector()
model.eval()

# Define Image Preprocessing
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Required for Xception
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalization
])

def predict_deepfake(image_path):
    image = Image.open(image_path).convert("RGB")  # Load Image
    image = transform(image).unsqueeze(0)  # Apply Transformations
    
    with torch.no_grad():
        output = model(image)
        probability = output.item()
    
    return probability  # Probability of being a deepfake

# Example Usage
image_path = "th (1).jpeg"  # Provide an image path
score = predict_deepfake(image_path)
print(f"Deepfake Probability: {score:.4f}")

Deepfake Probability: 0.5212


### FEATURE 2 - VIDEO UPLOAD WITH LIVE PROBABLITY 

In [9]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_rate=1):
    """
    Extracts frames from a video at a given frame rate.
    :param video_path: Path to the input video file
    :param output_folder: Folder to save extracted frames
    :param frame_rate: Extract 1 frame per 'frame_rate' seconds
    """
    os.makedirs(output_folder, exist_ok=True)  # Create output folder

    cap = cv2.VideoCapture(video_path)  # Load Video
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get frames per second
    frame_interval = fps * frame_rate  # Set frame extraction interval
    
    count = 0
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()  # Read Frame
        if not ret:
            break  # Stop if video ends
        
        if count % frame_interval == 0:
            frame_filename = f"{output_folder}/frame_{frame_id}.jpg"
            cv2.imwrite(frame_filename, frame)  # Save frame as an image
            frame_id += 1
        
        count += 1  # Update frame counter

    cap.release()
    cv2.destroyAllWindows()
    print(f"Extracted {frame_id} frames from {video_path}")

# Example Usage
video_path = "WhatsApp Video 2025-02-27 at 11.44.35_3f0b0dfd.mp4"  # Provide a video path
output_folder = "frames"
extract_frames(video_path, output_folder)

Extracted 21 frames from WhatsApp Video 2025-02-27 at 11.44.35_3f0b0dfd.mp4


In [11]:
import glob

def detect_deepfake_in_video(video_path, output_folder):
    """
    Extracts frames and detects deepfakes in a video.
    :param video_path: Path to video file
    :param output_folder: Folder to save frames and results
    """
    extract_frames(video_path, output_folder)  # Extract frames

    results = {}  # Store frame results

    for frame in sorted(glob.glob(f"{output_folder}/*.jpg")):
        probability = predict_deepfake(frame)  # Predict deepfake score
        results[frame] = probability
        print(f"{frame}: Deepfake Probability = {probability:.4f}")

    return results

# Example Usage
video_results = detect_deepfake_in_video("WhatsApp Video 2025-02-27 at 11.44.35_3f0b0dfd.mp4", "frames")

Extracted 21 frames from WhatsApp Video 2025-02-27 at 11.44.35_3f0b0dfd.mp4
frames\frame_0.jpg: Deepfake Probability = 0.4893
frames\frame_1.jpg: Deepfake Probability = 0.5387
frames\frame_10.jpg: Deepfake Probability = 0.4929
frames\frame_11.jpg: Deepfake Probability = 0.5172
frames\frame_12.jpg: Deepfake Probability = 0.4999
frames\frame_13.jpg: Deepfake Probability = 0.5006
frames\frame_14.jpg: Deepfake Probability = 0.5196
frames\frame_15.jpg: Deepfake Probability = 0.5103
frames\frame_16.jpg: Deepfake Probability = 0.5134
frames\frame_17.jpg: Deepfake Probability = 0.5318
frames\frame_18.jpg: Deepfake Probability = 0.5306
frames\frame_19.jpg: Deepfake Probability = 0.5291
frames\frame_2.jpg: Deepfake Probability = 0.5342
frames\frame_20.jpg: Deepfake Probability = 0.5344
frames\frame_3.jpg: Deepfake Probability = 0.5301
frames\frame_4.jpg: Deepfake Probability = 0.5154
frames\frame_5.jpg: Deepfake Probability = 0.5098
frames\frame_6.jpg: Deepfake Probability = 0.5004
frames\frame_

### FEATURE 3 -- LIVE WEBCAM DETECTION

In [16]:
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

# Load OpenCV's pre-trained face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_fake(frame, prev_frame):
    """ 
    Detects movement by comparing the current frame with the previous one.
    Returns a fake probability (0-100%).
    """
    if prev_frame is None:
        return 0  # No previous frame to compare
    
    diff = cv2.absdiff(frame, prev_frame)
    score = np.mean(diff)  # Calculate difference score

    probability = min(int(score / 2), 100)  # Scale to 0-100%
    return probability

def open_webcam():
    cap = cv2.VideoCapture(0)
    prev_frame = None  # Store previous frame

    def update_frame():
        nonlocal prev_frame
        ret, frame = cap.read()
        if not ret:
            return

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        # Convert frame to RGB (for displaying in Tkinter)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Compute deepfake probability
        probability = detect_fake(frame, prev_frame)
        prev_frame = frame.copy()  # Store frame for next comparison

        for (x, y, w, h) in faces:
            # Determine border color: Red (Fake) or Green (Real)
            color = (255, 0, 0) if probability > 50 else (0, 255, 0)

            # Draw a colored rectangle around the detected face
            cv2.rectangle(frame_rgb, (x, y), (x + w, y + h), color, 3)

            # Display probability percentage on the frame
            text = f"{probability}%"
            cv2.putText(frame_rgb, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Convert frame to Tkinter format
        img = Image.fromarray(frame_rgb)
        img_tk = ImageTk.PhotoImage(image=img)

        label.img_tk = img_tk
        label.config(image=img_tk)

        # Update label text
        status_text = f"Deepfake Probability: {probability}%"
        label_status.config(text=status_text, fg="red" if probability > 50 else "green")

        root.after(10, update_frame)  # Refresh every 10ms

    root = tk.Tk()
    root.title("Live Deepfake Detection")

    label = tk.Label(root)
    label.pack()

    label_status = tk.Label(root, text="", font=("Arial", 16, "bold"))
    label_status.pack()

    update_frame()  # Start webcam stream

    root.mainloop()
    cap.release()

# Run the function
open_webcam()

## FEATURE 4 -- CLASSIFICATION REPORT DOWNLOADED AS PDF

In [24]:
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
from fpdf import FPDF
import os
import datetime

# Load OpenCV's pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_fake(frame, prev_frame):
    """
    Detects movement by comparing the current frame with the previous one.
    Returns a fake probability (0-100%).
    """
    if prev_frame is None:
        return 0  # No previous frame to compare
    
    diff = cv2.absdiff(frame, prev_frame)
    score = np.mean(diff)  # Calculate difference score

    probability = min(int(score / 2), 100)  # Scale to 0-100%
    return probability

def process_frame(frame, prev_frame):
    """ Detects faces, applies colored borders, and returns processed frame with probability. """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    # Convert frame to RGB for display
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Compute deepfake probability
    probability = detect_fake(frame, prev_frame)

    for (x, y, w, h) in faces:
        # Choose color: Red (Fake) or Green (Real)
        color = (255, 0, 0) if probability > 50 else (0, 255, 0)

        # Draw rectangle around face
        cv2.rectangle(frame_rgb, (x, y), (x + w, y + h), color, 3)

        # Display probability percentage
        text = f"{probability}%"
        cv2.putText(frame_rgb, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    return frame_rgb, probability

def process_image():
    """ Processes an image for deepfake detection. """
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if not file_path:
        return
    
    frame = cv2.imread(file_path)
    if frame is None:
        print("Error: Could not read the image file!")
        return
    
    processed_frame, probability = process_frame(frame, None)
    
    # Save processed image
    output_path = "th (1).jpeg"
    cv2.imwrite(output_path, cv2.cvtColor(processed_frame, cv2.COLOR_RGB2BGR))
    print(f"Processed image saved as {output_path}")
    
    generate_pdf(file_path, probability)  # Generate PDF Report
    
    # Display image
    cv2.imshow("Processed Image", processed_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video():
    """ Processes a video file for deepfake detection. """
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
    if not file_path:
        return
    
    cap = cv2.VideoCapture(file_path)
    prev_frame = None
    
    # Video properties
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_path = "WhatsApp Video 2025-02-27 at 11.44.35_3f0b0dfd.mp4"
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    total_probability = 0
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame, probability = process_frame(frame, prev_frame)
        prev_frame = frame.copy()
        
        total_probability += probability
        frame_count += 1
        
        out.write(cv2.cvtColor(processed_frame, cv2.COLOR_RGB2BGR))  # Save frame
        
        # Show frame
        cv2.imshow("Processed Video", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    average_probability = total_probability // frame_count if frame_count else 0
    generate_pdf(file_path, average_probability)
    print(f"Processed video saved as {output_path}")

def generate_pdf(file_name, confidence_score):
    """ Generates a PDF report with confidence score. """
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=16)
    pdf.cell(200, 10, "Deepfake Detection Report", ln=True, align='C')
    pdf.ln(10)
    
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, f"File: {os.path.basename(file_name)}", ln=True)
    pdf.cell(200, 10, f"Confidence Score: {confidence_score}%", ln=True)
    pdf.cell(200, 10, f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)
    
    output_pdf = "deepfake_report.pdf"
    pdf.output(output_pdf)
    print(f"Report saved as {output_pdf}")

# === MAIN MENU ===
root = tk.Tk()
root.title("Deepfake Detector")
root.geometry("400x300")

tk.Label(root, text="Choose an option:", font=("Arial", 14)).pack(pady=10)

tk.Button(root, text="Process Image", command=process_image, font=("Arial", 12)).pack(pady=5)
tk.Button(root, text="Process Video", command=process_video, font=("Arial", 12)).pack(pady=5)

tk.Label(root, text="Processed files and PDF reports will be saved.", font=("Arial", 10)).pack(pady=10)

root.mainloop()

Processed image saved as th (1).jpeg
Report saved as deepfake_report.pdf
Report saved as deepfake_report.pdf
Processed video saved as WhatsApp Video 2025-02-27 at 11.44.35_3f0b0dfd.mp4


## FEATURE - 5 alert mechanism based on deepfake probability

In [33]:
import cv2
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
from playsound import playsound

# Load OpenCV's pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_fake(frame, prev_frame):
    """ Detects movement and calculates deepfake probability. """
    if prev_frame is None:
        return 0  # No comparison possible

    diff = cv2.absdiff(frame, prev_frame)
    score = np.mean(diff)  # Calculate difference score

    probability = min(int(score / 2), 100)  # Scale to 0-100%
    return probability

def process_frame(frame, prev_frame):
    """ Detects faces and applies deepfake probability calculation. """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for display
    probability = detect_fake(frame, prev_frame)

    for (x, y, w, h) in faces:
        color = (255, 0, 0) if probability > 50 else (0, 255, 0)
        cv2.rectangle(frame_rgb, (x, y), (x + w, y + h), color, 3)
        cv2.putText(frame_rgb, f"{probability}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Play alert sound if deepfake probability is high
        if probability > 50:
            playsound("alert.mp3", block=False)

    return frame_rgb, probability

def open_webcam():
    """ Opens webcam and performs real-time deepfake detection with sound alerts. """
    cap = cv2.VideoCapture(0)
    prev_frame = None

    def update_frame():
        nonlocal prev_frame
        ret, frame = cap.read()
        if not ret:
            return

        frame, probability = process_frame(frame, prev_frame)
        prev_frame = frame.copy()

        img = Image.fromarray(frame)
        img_tk = ImageTk.PhotoImage(image=img)
        label.img_tk = img_tk
        label.config(image=img_tk)

        label_status.config(text=f"Deepfake Probability: {probability}%", fg="red" if probability > 50 else "green")
        root.after(10, update_frame)

    root = tk.Tk()
    root.title("Deepfake Detector with Audio Alert")

    label = tk.Label(root)
    label.pack()

    label_status = tk.Label(root, text="", font=("Arial", 16, "bold"))
    label_status.pack()

    update_frame()
    root.mainloop()
    cap.release()

# Run the webcam function
open_webcam()

In [41]:
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import winsound  # Built-in module for beep sound (Windows only)

def play_beep():
    """Plays a simple beep sound."""
    winsound.Beep(1000, 500)  # Frequency = 1000 Hz, Duration = 500 ms

def open_image():
    """Opens a file dialog to select an image and displays it."""
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpeg;*.jpg;*.png")])
    if not file_path:
        return  # If no file is selected, do nothing

    # Play beep sound
    play_beep()

    # Load image using OpenCV and convert it to RGB
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert to Tkinter format and display
    img = Image.fromarray(image)
    img_tk = ImageTk.PhotoImage(img)

    label.config(image=img_tk)
    label.image = img_tk  # Keep reference to avoid garbage collection

def exit_program(event):
    """Closes the application when 'q' is pressed."""
    root.quit()

# Initialize Tkinter
root = tk.Tk()
root.title("Image Upload with Beep Sound")

# Create GUI components
btn_upload = tk.Button(root, text="Upload Image", command=open_image, font=("Arial", 14))
btn_upload.pack(pady=10)

label = tk.Label(root)
label.pack()

# Bind 'q' key to exit the application
root.bind("<q>", exit_program)

root.mainloop()